In [1]:
import pandas as pd
import numpy as np
import os

import psycopg2
conn = psycopg2.connect(dbname='churn', 
                        user='postgres',
                        password='postgres',
                        host='bi-assess.cvtbaved9qqg.us-west-2.rds.amazonaws.com',
                        port=5432)

In [2]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None, 'display.max_columns', None)

In [3]:
dim_account = os.path.join('data', 'dim_account', '000')
dim_calendar = os.path.join('data', 'dim_calendar', '000')
dim_product = os.path.join('data', 'dim_product', '000')
dim_territory = os.path.join('data', 'dim_territory', '000')
fact_sales_revenue = os.path.join('data', 'fact_sales_revenue', '000')

In [4]:
# 0 means open
conn.closed

0

In [5]:
create_dim_account = """
    CREATE TABLE dim_account (
        payer_account_id TEXT, 
        payer_account_company_name TEXT,
        payer_account_registration_date TIMESTAMP WITHOUT TIME ZONE,
        payer_account_status_code TEXT,
        payer_account_role_name TEXT,
        payer_account_creation_date TIMESTAMP WITHOUT TIME ZONE,
        payer_account_last_update_date TIMESTAMP WITHOUT TIME ZONE,
        payer_account_registration_tax_country_code TEXT,
        is_payer_account_bill_in_90_days CHAR(1),
        is_payer_account_enterprise CHAR(1),
        is_payer_account_domain_free_domain CHAR(1),
        is_payer_account_on_enterprise_support CHAR(1),
        is_payer_account_on_business_support CHAR(1),
        is_payer_account_fraud CHAR(1),
        is_payer_account_suspended CHAR(1),
        is_payer_account_internal CHAR(1),
        is_payer_account_inp_terminated CHAR(1),
        is_payer_account_on_behalf_of CHAR(1),
        is_payer_account_tax_exempt CHAR(1),
        is_payer_account_reseller CHAR(1),
        payer_account_first_billing_reporting_date TIMESTAMP WITHOUT TIME ZONE,
        payer_account_first_payment_reporting_date TIMESTAMP WITHOUT TIME ZONE,
        payer_account_first_usage_date TIMESTAMP WITHOUT TIME ZONE
    );
"""

create_dim_calendar = """
    CREATE TABLE dim_calendar (
        calendar_sid INTEGER,
        calendar_date DATE,
        day_of_week_name TEXT,
        day_of_week_number INTEGER,
        day_of_week_code TEXT,
        day_of_quarter_number INTEGER,
        day_of_year_number INTEGER,
        calendar_period TEXT,
        week_id INTEGER,
        prior_week_id INTEGER,
        subsequent_week_id INTEGER,
        week_number INTEGER,
        week_code TEXT,
        month_id INTEGER,
        prior_month_id INTEGER,
        subsequent_month_id INTEGER,
        month_number INTEGER,
        month_code TEXT,
        month_num_of_days INTEGER,
        year_id INTEGER,
        prior_year_id INTEGER,
        subsequent_year_id INTEGER,
        year_number INTEGER,
        year_code TEXT,
        ytd_flag BOOLEAN,
        yoy_flag BOOLEAN,
        report_month_flag BOOLEAN,
        prospecting_day_flag BOOLEAN
    )
"""

create_dim_product = """
    CREATE TABLE dim_product (
        product_sid TEXT,
        product_line_name TEXT,
        product_name TEXT,
        sub_product_name TEXT,
        service_group_name TEXT,
        product_companyn_name TEXT
    )
"""

create_dim_territory = """
    CREATE TABLE dim_territory (
        territory_sid TEXT,
        region TEXT,
        sub_region TEXT,
        territory_code TEXT,
        segment TEXT
    )
"""

create_fact_sales_revenue = """
    CREATE TABLE fact_sales_revenue (
       territory_sid TEXT,
       payer_account_id TEXT,
       month_id INTEGER,
       product_sid TEXT,
       sales_revenue NUMERIC(38,6)
    )
"""

In [6]:
tables = [
#    'dim_account',
    'dim_calendar',
    'dim_product',
    'dim_territory',
#    'fact_sales_revenue'
]

In [7]:
try:
    cur = conn.cursor()
    cur.execute('SELECT now()')
    data = cur.fetchall()
    print(data)
except psycopg2.OperationalError as oe:
    print(oe)

[(datetime.datetime(2019, 4, 16, 7, 11, 37, 402107, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),)]


In [8]:
def create_table(tablename):
    try:
        cur = conn.cursor()
        func_name = 'create_' + tablename
        cur.execute(eval(func_name))
        conn.commit()
        print('Table ' + tablename + ' created.')
    except psycopg2.OperationalError as oe:
        print(oe)

In [9]:
def copy_from_file(filepath, tablename):
    try:
        cur = conn.cursor()
        f = open(filepath, 'r')
        cur.copy_from(f, tablename, sep=',')
        conn.commit()
        f.close()
        print(filepath + ' copied into table ' + tablename + '.')
    except psycopg2.OperationalError as oe:
        print(oe)

In [10]:
for table in tables:
    create_table(table)
    copy_from_file(eval(table), table)

Table dim_calendar created.
data\dim_calendar\000 copied into table dim_calendar.
Table dim_product created.
data\dim_product\000 copied into table dim_product.
Table dim_territory created.
data\dim_territory\000 copied into table dim_territory.
